# 계절성지수 산출


---

---

In [ ]:
# 라이브러리 선언
import pandas as pd
import numpy as np

In [ ]:
# csv 파일 불러오기
selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

# A. 데이터 전처리

### A-1. 타입 통합 & 불량 데이터 처리
- 타입통합
- QTY컬럼→실수(float), 이외컬럼→문자(str)로 변경

ppt28p문제
[데이터타입 통합]
kopo_channel_seasonality_new.csv 파일을 불러온 후
selloutData 변수에 담으세요
이후 QTY컬럼→실수(float), 이외컬럼→문자(str)로 변경하세요

In [3]:
selloutData.dtypes
# QTY컬럼→실수(float), 이외컬럼→문자(str)로 변경

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
dtype: object

In [4]:
#1.
selloutData.YEARWEEK=selloutData.YEARWEEK.astype(str)
#selloutData["YEARWEEK"] = selloutData["YEARWEEK"].astype(str)

In [5]:
#2
selloutData.columns
selloutData = selloutData.astype({'REGIONID':str, 'PRODUCT':str, 'YEARWEEK':str, 'QTY':float})
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK     object
QTY         float64
dtype: object

- [불량 데이터 처리]
- kopo_channel_seasonality_new.csv 자료를 담은
selloutData 변수에서
QTY컬럼 음수(반품)인 경우 0, 양수인 경우 기존 QTY 값
유지 -> QTY_NEW 컬럼 추가

In [6]:
selloutData.describe()  #min -3.640000e+02

#음수인경우 count
selloutData.loc[selloutData.QTY <0].shape #(323,4) 1.나
selloutData[selloutData.QTY < 0]["QTY"].count() #2. .count() 2.ppt
selloutData["QTY_NEW"] = np.where(selloutData.QTY<0, 0, selloutData.QTY)
# #확인
selloutData.loc[selloutData["QTY_NEW"] < 0]
selloutData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW
0,A60,PRODUCT4,201402,71.0,71.0
1,A60,PRODUCT59,201402,22275.0,22275.0
2,A60,PRODUCT34,201402,4463.0,4463.0
3,A60,PRODUCT47,201402,0.0,0.0
4,A60,PRODUCT56,201402,23.0,23.0
...,...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0,824.0
124654,A10,PRODUCT56,201630,275.0,275.0
124655,A10,PRODUCT61,201630,0.0,0.0
124656,A10,PRODUCT12,201630,15021.0,15021.0


### A-2. 데이터 통합
-  연도별 주차수가 다른 Year가 관찰됨
- 이유: 1년이 365(366)일이므로
필연적으로 53주차가 때때로 발생
- 영향: 주차별 평균 계절성지수 산출
시 주차수가 통일되지 않음
- 해결: 53주차 데이터를 제거함

[데이터 통합]
selloutData 자료에서
YEAR, WEEK 컬럼을 생성하고 WEEK 가 52 이하인
데이터만 조회한 후 refinedSelloutData 변수에 담음

In [7]:
selloutData["YEAR"] = selloutData["YEARWEEK"].str[0:4]
selloutData["WEEK"] = selloutData["YEARWEEK"].str[4:]
# selloutData.loc[selloutData['WEEK'].astype(int) > 52].shape #794 #지금 거의다 object =str이라서 astype으로 이때만 잠깐 바꿔준다
# selloutData.loc[selloutData['WEEK'].astype(int) > 52]['WEEK'].count() #794 53주차가 794개 있다.

refinedSelloutData = selloutData.loc[selloutData['WEEK'].astype(int) <= 52]
refinedSelloutData

#확인1
refinedSelloutData.loc[refinedSelloutData['WEEK'].astype(int) > 52] #여기도 astype으로 잠깐 바꿔줘야한다.
#정렬 으로 확인2
refinedSelloutData.sort_values(by="WEEK", ascending=False) #이거 아직 반영안된것임 refinedSelloutData =  해야 반영됨.

refinedSelloutData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,02
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,02
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,02
3,A60,PRODUCT47,201402,0.0,0.0,2014,02
4,A60,PRODUCT56,201402,23.0,23.0,2014,02
...,...,...,...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0,824.0,2016,30
124654,A10,PRODUCT56,201630,275.0,275.0,2016,30
124655,A10,PRODUCT61,201630,0.0,0.0,2016,30
124656,A10,PRODUCT12,201630,15021.0,15021.0,2016,30


# B. 추세선 도출
### 대표값 생성
-  지역/상품/연도별 판매량 산출
=> 지역/상품/연도별 판매량 산출
(Group by 활용)
- 소스데이터와 키 (지역,상품,연도) 키로
조인한다.

---

refinedData 에서 →
지역, 상품, 연주차 단위로 정렬하여
sortedData 변수에 담음

In [8]:
sortKey = ['REGIONID','PRODUCT','YEARWEEK']
sortedData=refinedSelloutData.sort_values(by=sortKey, ascending=[True, True, True])
sortedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
298,A00,PRODUCT34,201401,661.0,661.0,2014,01
1757,A00,PRODUCT34,201402,679.0,679.0,2014,02
3125,A00,PRODUCT34,201403,578.0,578.0,2014,03
205,A00,PRODUCT34,201404,532.0,532.0,2014,04
4369,A00,PRODUCT34,201405,516.0,516.0,2014,05
...,...,...,...,...,...,...,...
102292,A77,PRODUCT12,201648,4152.0,4152.0,2016,48
97525,A77,PRODUCT12,201649,5086.0,5086.0,2016,49
102329,A77,PRODUCT12,201650,5846.0,5846.0,2016,50
101211,A77,PRODUCT12,201651,4933.0,4933.0,2016,51


[지역, 상품, 연도별 집계] *groupby

sortedData 에서 지역, 상품 단위
판매량(QTY_NEW) 의 평균 연산 후
groupData 변수에 담음

이후 컬럼명을 QTY_MEAN로 변경

In [9]:
groupKey = ["REGIONID","PRODUCT","YEAR"]

In [ ]:
#나

groupData = sortedData.groupby(groupKey)['QTY_NEW'].agg(['mean']) #여기서
groupData=groupData.reset_index().rename(columns = {'mean':'QTY_MEAN'})  #인덱스를 리셋한다.
groupData

*join
refinedSelloutData와 groupData를
[REGIONID, PRODUCT, YEAR] 키로 조인하여
mergedData 변수에 담음

In [13]:
joinKey = ["REGIONID","PRODUCT","YEAR"]
mergedData = pd.merge(left = refinedSelloutData, right=groupData, left_on=joinKey, right_on=joinKey, how="left")
mergedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN
0,A60,PRODUCT4,201402,71.0,71.0,2014,02,29.134615
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,02,22993.923077
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,02,6077.653846
3,A60,PRODUCT47,201402,0.0,0.0,2014,02,0.288462
4,A60,PRODUCT56,201402,23.0,23.0,2014,02,16.730769
...,...,...,...,...,...,...,...,...
123859,A10,PRODUCT60,201630,824.0,824.0,2016,30,721.923077
123860,A10,PRODUCT56,201630,275.0,275.0,2016,30,235.307692
123861,A10,PRODUCT61,201630,0.0,0.0,2016,30,0.019231
123862,A10,PRODUCT12,201630,15021.0,15021.0,2016,30,29025.711538


## C. 계절성 지수 산출

계절성 지수 계산
계절성 지수를 산출한다.
=> (계절성 지수) = (실 qty)/(지역/상품/연도별 판매량)

#QTY 원본.but 음수있음
#QTY_NEW는 음수값 제거한거
#QTY_MEAN은 그룹바이로 평균 구한거

SO.계절성지수 = QTY_NEW / QTY_MEAN

* 계절성지수 컬럼 생성 SEASONALITY

In [14]:
mergedData['SEASONALITY'] = mergedData['QTY_NEW'] / mergedData['QTY_MEAN']
mergedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN,SEASONALITY
0,A60,PRODUCT4,201402,71.0,71.0,2014,02,29.134615,2.436964
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,02,22993.923077,0.968734
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,02,6077.653846,0.734329
3,A60,PRODUCT47,201402,0.0,0.0,2014,02,0.288462,0.000000
4,A60,PRODUCT56,201402,23.0,23.0,2014,02,16.730769,1.374713
...,...,...,...,...,...,...,...,...,...
123859,A10,PRODUCT60,201630,824.0,824.0,2016,30,721.923077,1.141396
123860,A10,PRODUCT56,201630,275.0,275.0,2016,30,235.307692,1.168683
123861,A10,PRODUCT61,201630,0.0,0.0,2016,30,0.019231,0.000000
123862,A10,PRODUCT12,201630,15021.0,15021.0,2016,30,29025.711538,0.517507


In [15]:
mergedData.groupby(by=joinKey)['SEASONALITY'].agg(['mean'])

mean
REGIONID PRODUCT   YEAR      
A00      PRODUCT34 2014   1.0
                   2015   1.0
                   2016   1.0
         PRODUCT58 2014   1.0
                   2015   1.0
...                       ...
A77      PRODUCT1  2015   1.0
                   2016   1.0
         PRODUCT12 2014   1.0
                   2015   1.0
                   2016   1.0

[2382 rows x 1 columns]

In [16]:
finalResult = mergedData.groupby(by=['REGIONID','PRODUCT','WEEK'])['SEASONALITY'].agg(['mean']).reset_index()

In [17]:
finalResult.rename(columns={'mean':'SEASONALITY'}, inplace=True)
finalResult

,REGIONID,PRODUCT,WEEK,SEASONALITY
0,A00,PRODUCT34,01,1.570782
1,A00,PRODUCT34,02,1.755540
2,A00,PRODUCT34,03,1.319460
3,A00,PRODUCT34,04,1.490298
4,A00,PRODUCT34,05,1.061909
...,...,...,...,...
41283,A77,PRODUCT12,48,1.352712
41284,A77,PRODUCT12,49,1.094083
41285,A77,PRODUCT12,50,1.386116
41286,A77,PRODUCT12,51,1.255192
